# Khai báo thư viện và các biến

In [ ]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd
import threading
#Khai báo chủ đề và danh mục cần lấy
topics = ['thoi-su','the-gioi','kinh-te','doi-song','suc-khoe','gioi-tre','giao-duc','van-hoa','giai-tri','the-thao',
          'cong-nghe','xe','du-lich']
cac_danh_muc = ['Thời sự',
                'Thế giới',
                'Kinh tế',
                'Đời sống',
                'Sức khỏe',
                'Giới trẻ',
                'Giáo dục',
                'Văn hóa',
                'Giải trí',
                'Thể thao',
                'Công nghệ',
                'Xe',
                'Du lịch']
#Khai báo biến cần thiết
base_url = 'https://thanhnien.vn'
max_links = 30000 #tổng số lượng link lấy tối đa
max_danhmuc = 1000 #số lượng tối đa mỗi danh mục trên tổng link
links = [] #lưu trữ link
count_danhmuc = {} #đếm số lượng danh mục đã lấy
data = []
lock = threading.Lock()

# Hàm đa luồng

In [ ]:
#hàm thread
def da_luong(ham, link):
    threads = []
    thread = threading.Thread(target=ham, args=(link,))
    thread.start()
    threads.append(thread)

# Lấy link

## Hàm lấy link

In [ ]:
def get_link(url):
    print(len(links))
    #Kiểm tra nếu lỗi thì bỏ qua
    try:
        response = rq.get(f"{base_url}{url}")
        if response.status_code == 200:
            soup = BeautifulSoup(response.content,"html.parser")
            #Tìm all các ô có đường dẫn
            content = soup.find_all('a',class_='box-category-link-title')
            # Lấy link bài viết ở trang
            for article in content:
                with lock:
                    #Kiểm tra href này có trong links chưa nếu chưa thì thêm vào
                    if article['href'] not in links:
                        links.append(article['href'])
                        # print(article['href'])
        else:
            print("Không thể truy cập trang web.")
    except:
        return

## Tiến hành lấy các link

In [ ]:
#Lặp qua các topic lấy các href
for topic in topics:
    get_link("/"+topic)
#Dùng đa luồng lặp lấy các href từ links
for link in links:
    #Nếu số lượng link lớn hơn 30000 dừng
    if len(links) > max_links:
        break
    da_luong(get_link, link)

Streaming output truncated to the last 5000 lines.
20951
20951
20951
20951
20951
20951
20951
20951
20951
20951
2095120951
20951

20951
20951
20951
20951
20951
2095120951

20951
20953
20954
20956
20956
20958
20958
20959
20962
20967
20967
20968
20968
20969
20972
20975
20975
2097520975

20977
20988
20989
20989
20992
20997
21001
21004
21010
21010
21011
21014
21014
21014
21015
21015
21021
21027
21032
21038
21042
21047
21052
Không thể truy cập trang web.
21058
21062
21069
21072
21073
21076
21077
21081
21084
21091
21091
21092
21102
21106
21106
21108
21108
21112
21115
21115
21115
Không thể truy cập trang web.
21120
21123
21123
21127
21128
21131
21131
21135
21138
21142
21150
21153
21157
21159
21161
21164
21171
21173
21173
21179
21179
21180
21188
21188
21188
21188
21188
21188
21188
21188
21188
21188
21188
21188
21188
21188
21188
21188
21188
21188
2118821188

21188
21189
21189
21189
2118921189

21189
21189
21189
21189
2118921189

21189
21189
21189
21189
21189
21189
21189
21189
21189
21189
21189
2

In [ ]:
print('Số lượng link đã crawl: ',len(links))

Số lượng link đã crawl:  30008


## Lưu các link đã crawl để backup

In [ ]:
#Lưu thành file txt
with open('/content/output_links.txt','w') as file:
    for link in links:
        file.write(link+'\n')

# Lấy content

## Hàm lấy content

In [ ]:
def get_content(url):
    print(len(data))
    try:
        html_sub = rq.get(base_url+url)
        # Use BeautifulSoup to parse the HTML
        soup_sub = BeautifulSoup(html_sub.content, 'html.parser')
        if soup_sub.find('div',class_ = 'detail-cate'):
        #Lấy danh mục
            danh_muc = soup_sub.find('div',class_ = 'detail-cate').find('a').text.strip()
            if danh_muc not in cac_danh_muc:
                return
            #Kiểm tra xem danh mục đã đủ số lượng chưa
            if count_danhmuc.get(danh_muc, 0) >= max_danhmuc:
                    print(f"Đã đủ số lượng dữ liệu cho nhãn: {danh_muc}")
                    return
            #Lấy nội dung của bài viết
            noi_dung = soup_sub.find('div',class_='detail-cmain').get_text().strip()
            #Lưu danh mục và nội dung dưới dạng từ điển
            with lock:
                data.append({'Danh mục':danh_muc,'Nội dung':noi_dung})
                #Tăng số lượng danh mục
                count_danhmuc[danh_muc] = count_danhmuc.get(danh_muc,0) + 1
        else:
            return
    except:
        return

## Tiến hành lấy content từ các link đã crawl

In [ ]:
#Dùng đa luồng lấy danh mục và nội dung từ các link đã crawl
for link in links:
    da_luong(get_content, link)

Streaming output truncated to the last 5000 lines.

1208012080

12080
12080
12080
12080
12080
12080
12080
1208012080

12080
Đã đủ số lượng dữ liệu cho nhãn: Văn hóa
12080
12080
1208012080
1208012080


12080
12080
12080
12080
12080
12080
12080
12080
12080
12080
12080
12080
12080
12080
12080
12080
12080
1208012080

12080
12080
12080
12080
1208012080

12080
12080
12080
1208012080
12080

12080
12080
12080
12080
12080
12080
12080
12080
12080
12080
12080
1208012080

12080
12080
12080
1208012080
12080

12080
12080
12080
12080
12080
12080
12080
12080
12080
12080
12080
12080
12080
12080
12080
12080
12080
12080
Đã đủ số lượng dữ liệu cho nhãn: Văn hóa
Đã đủ số lượng dữ liệu cho nhãn: Thể thaoĐã đủ số lượng dữ liệu cho nhãn: Văn hóa
Đã đủ số lượng dữ liệu cho nhãn: Giới trẻĐã đủ số lượng dữ liệu cho nhãn: Văn hóa
12080
Đã đủ số lượng dữ liệu cho nhãn: Đời sống

Đã đủ số lượng dữ liệu cho nhãn: Công nghệ
Đã đủ số lượng dữ liệu cho nhãn: Văn hóa
Đã đủ số lượng dữ liệu cho nhãn: Thể thao
Đã đủ số lư

In [ ]:
print('Số lượng dữ liệu:',len(data))

Số lượng dữ liệu: 12863


In [ ]:
print('Số lượng link chưa được thu thập:',len(links)-len(data))

Số lượng link chưa được thu thập: 17145


# Lưu dữ liệu

In [ ]:
# Lưu data thành file csv
df = pd.DataFrame(data)
df.to_csv('/content/Data.csv', index=False)